In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("..")
from plot_OpSims import get_metric_medians
from script_utils import get_opsim_areas

In [3]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [4]:
# import convenience functions
import sys 
sys.path.append("../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

## Load the results
We first load the results of the run into memory. We will read jointly the FBS v1.5 and FBS v1.6 results, and will differentiate at the time of plotting.

In [5]:
NSIDE = 64
your_username = "rjassef"
folder_mafoutput = "DDFs_EM5_depths_{0:d}".format(NSIDE)
resultDbPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [6]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [7]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSID

In [8]:
# check keys
dbRuns = list(bundleDicts_raw.keys())
print(bundleDicts_raw[dbRuns[0]].keys())

dict_keys([(1, 'ExgalM5_with_cuts_AGN_ECDFS_u'), (2, 'ExgalM5_with_cuts_AGN_XMM-LSS_g'), (3, 'ExgalM5_with_cuts_AGN_ECDFS_g'), (4, 'ExgalM5_with_cuts_AGN_AllDDFs_u'), (5, 'ExgalM5_with_cuts_AGN_AllDDFs_g'), (6, 'ExgalM5_with_cuts_AGN_COSMOS_u'), (7, 'ExgalM5_with_cuts_AGN_ELAISS1_u'), (8, 'ExgalM5_with_cuts_AGN_EDFS_u'), (9, 'ExgalM5_with_cuts_AGN_XMM-LSS_u'), (10, 'ExgalM5_with_cuts_AGN_COSMOS_g'), (11, 'ExgalM5_with_cuts_AGN_EDFS_g'), (12, 'ExgalM5_with_cuts_AGN_ELAISS1_g')])


## Process the metric results for each OpSim and save the results in a table.

Note that in general, the rank will be from worst (0%) to best (100%)

In [9]:
mds = dict()
rank_sort = dict()
total_areas = dict()

### Function to pull out a specific DDF from the bundleDicts 

In [10]:
def get_bundleDicts_use(filter, DDF_name, bundleDicts_raw):
    bundleDicts_use = dict()
    metricName = 'ExgalM5_with_cuts_AGN_{0}_{1}'.format(DDF_name, filter)
    Key = (1, metricName)
    for run in dbRuns:
        for raw_key in bundleDicts_raw[run].keys():
            if raw_key[1]==metricName:
                #mask = bundleDicts_raw[run][raw_key].metricValues.mask
                #data = bundleDicts_raw[run][raw_key].metricValues.data[~mask]
                #data = data[~(np.isnan(data) | np.isinf(data))]
                bundleDicts_use[run] = dict()
                bundleDicts_use[run][Key] = bundleDicts_raw[run][raw_key]
                break
    return Key, bundleDicts_use

#### u-band, 3-sigma depth

In [11]:
def mlim_3sigma(mlim_5sigma):
    return mlim_5sigma + 2.5*np.log10(5./3.)

In [12]:
DDFs = ['AllDDFs', 'ECDFS', 'COSMOS', 'XMM-LSS', 'ELAISS1', 'EDFS']
filters = ["u","g"]

for filter in filters:
    print(filter)
    mds[filter] = dict()
    rank_sort[filter] = dict()
    total_areas[filter] = dict()

    for DDF in DDFs:

        #Create the bundleDict appropriate for this DDF.
        Key = tuple()
        bundleDicts_use = dict()
        Key, bundleDicts_use = get_bundleDicts_use(filter, DDF, bundleDicts_raw)

        data_func=None
        if filter=="u":
            data_func = mlim_3sigma
        
        mds[filter][DDF] = get_metric_medians(Key, bundleDicts_use, data_func=data_func)
        rank_sort[filter][DDF] = np.argsort(mds[filter][DDF])
        total_areas[filter][DDF] = get_opsim_areas(Key, bundleDicts_use)

        runs = list(bundleDicts_use.keys())
        print(DDF)
        print(runs[rank_sort[filter][DDF][ 0]], mds[filter][DDF][rank_sort[filter][DDF][0]], total_areas[filter][DDF][rank_sort[filter][DDF][0]])
        print(runs[rank_sort[filter][DDF][-1]], mds[filter][DDF][rank_sort[filter][DDF][-1]], total_areas[filter][DDF][rank_sort[filter][DDF][-1]])

u
AllDDFs
barebones_nexp2_v1.6_10yrs 26.78845702126506 107.42958658702935
ddf_heavy_v1.6_10yrs 28.09876828878882 109.10817387745168
ECDFS
barebones_nexp2_v1.6_10yrs 26.780999740328433 18.464460194645667
ddf_heavy_v1.6_10yrs 28.316801407607286 18.464460194645667
COSMOS
dm_heavy_nexp2_v1.6_10yrs 26.72749968991747 29.375277582390837
ddf_heavy_v1.6_10yrs 27.93756150912226 18.464460194645667
XMM-LSS
barebones_nexp2_v1.6_10yrs 26.781613848415112 18.464460194645667
ddf_heavy_v1.6_10yrs 27.936370282166877 19.303753839856835
ELAISS1
footprint_4_v1.710yrs 26.92214445978002 18.464460194645667
ddf_heavy_v1.6_10yrs 28.161143944570874 19.303753839856835
EDFS
ddf_dither2.00_v1.7_10yrs 26.589317205585527 84.76865816632784
ddf_heavy_v1.6_10yrs 28.206701520379504 33.57174580844667
g
AllDDFs
agnddf_v1.5_10yrs 27.1800997746149 108.26888023224052
ddf_heavy_v1.6_10yrs 28.85738487002459 109.10817387745168
ECDFS
agnddf_v1.5_10yrs 27.327688158085913 18.464460194645667
ddf_heavy_v1.6_10yrs 29.067986226252046 18

#### Save the table

In [13]:
for DDF in DDFs: 

    for filter in ["u","g"]:
    
        fname = "Table_DDF_{0}_band_depth.{1}.txt".format(filter, DDF)
        tab_file = open(fname,"w")
    
        Key = tuple()
        bundleDicts_use = dict()
        Key, bundleDicts_use = get_bundleDicts_use(filter, DDF, bundleDicts_raw)

        for k,run in enumerate(list(bundleDicts_use.keys())):
            tab_file.write("{0:50s}".format(run))
    
            rank_percent = ( (np.argwhere(k==rank_sort[filter][DDF])[0,0]+1) / 
                            len(mds[filter][DDF]) * 100.)
            tab_file.write(" {0:10.2f} {1:5.1f} {2:7.2f}".format(mds[filter][DDF][k], 
                                                            rank_percent,
                                                            total_areas[filter][DDF][k]))
            tab_file.write("\n")
        tab_file.close()